In [15]:
import json
import os
import time
from dotenv import load_dotenv
from openai import OpenAI


load_dotenv("../saved_keys.env")

assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please sign up for access to the OpenAI API and provide access token in keys.env file"

In [16]:
# Initialize the OpenAI client
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)

In [17]:
# Load the training data, you can create the training data with the generate_fine_tuning_dataset.py script
with open("training_data.jsonl", "r") as f:
    training_data = [json.loads(line) for line in f]


In [26]:
# Let's infer the first sample
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=training_data[0]["messages"][:2]
)
print(f"Input: {training_data[0]['messages'][:2]}")
print("-"*100)
print(f"Output: {response.choices[0].message.content}")


Input: [{'role': 'system', 'content': 'You are a helpful assistant converting notes to professional emails.'}, {'role': 'user', 'content': 'urgent: need meeting minutes by {deadline}\nfor stakeholder presentaton\ndetails:\n- This is needed for our upcoming presentation to stakeholders.\n- will need ur help asap\n- send 2 me when rdy Style: request'}]
----------------------------------------------------------------------------------------------------
Output: Subject: Request for Meeting Minutes by {Deadline} for Stakeholder Presentation

Dear [Recipient],

I hope this message finds you well. I am reaching out to request your urgent assistance in preparing meeting minutes for our recent meeting, which are required for our upcoming stakeholder presentation.

The meeting minutes are crucial for ensuring that we accurately represent the discussions and decisions made during the meeting to our stakeholders. Your prompt attention to this matter is greatly appreciated. 

Please send the comple

### Observation:

The model is able to handle the task, but it fails on the correct placeholder tags or isn't hitting the correct tone you prefer. Let's fix those issues by fine-tuning a model.


In [12]:
# Upload the training file
training_file = client.files.create(
    file=open("training_data.jsonl", "rb"),
    purpose="fine-tune"
)

In [13]:

# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo"  # Base model to fine-tune
)


Job status: validating_files


In [19]:
# Continuously check the status of the fine-tuning job
while True:
    job_status = client.fine_tuning.jobs.retrieve(job.id)
    print(f"Job status: {job_status.status}")

    if job_status.status in ['succeeded', 'failed']:
        break

    print("Waiting 10 seconds...")
    time.sleep(10)

if job_status.status == 'succeeded':
    print(f"Fine-tuning complete! You can now use model: {job_status.fine_tuned_model}")
else:
    print("Fine-tuning failed. Check the job status for more information.")

# Once the job is complete, you can use the fine-tuned model
# The fine-tuned model ID will be available in job_status.fine_tuned_model

Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 seconds...
Job status: running
Waiting 10 sec

In [25]:
# Now we can use the fine-tuned model to generate the email with training_data[0]['messages']
# Generally it is ad practise to test the model with a sample input from the training data,
# but we want to check the different outputs between the generic gpt-3.5-turbo and the fine-tuned model.

# Test the fine-tuned model with a sample input training_data[0]['messages']
completion = client.chat.completions.create(
    model=job_status.fine_tuned_model,  # Use the fine-tuned model
    messages=training_data[0]['messages'][:2]
)

# Print the generated response
print("Generated email:")
print(completion.choices[0].message.content)

Generated email:
Subject: Request for meeting minutes

Body: Hi {name},

I hope you're doing well. I'm reaching out because I need meeting minutes by {deadline}.

This is needed for our upcoming presentation to stakeholders.

Could you please help me with this? Let me know if you need any additional information.

Thank you in advance for your help.

Best,
{name}


In [20]:
# Now we can use the fine-tuned model to generate the email

# Test the fine-tuned model with a sample input
completion = client.chat.completions.create(
    model=job_status.fine_tuned_model,  # Use the fine-tuned model
    messages=[
        {"role": "system", "content": "You are a helpful assistant converting notes to professional emails."},
        {"role": "user", "content": "schedule a meeting with John about the Q4 planning"}
    ]
)

# Print the generated response
print("Generated email:")
print(completion.choices[0].message.content)


Generated email:
Subject: Request for meeting

Body: Dear John,

I hope you're doing well. I'm reaching out because I'd like to discuss Q4 planning with you.

Please let me know when you're available to meet.

Thank you,
{name}
